In [1]:
!pip install tf-slim
import os
import numpy as np
from PIL import Image
from tensorflow.compat.v1 import disable_v2_behavior, placeholder
import tf_slim as slim
from tf_slim.nets import inception
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
import cv2


In [2]:
import zipfile
import os
with zipfile.ZipFile("resized_data (1).zip", 'r') as zip_ref:
    zip_ref.extractall("/content/resized_data")

In [3]:
!pip install tensorflow

  Using cached tensorflow_intel-2.17.0-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
  Using cached tensorboard-2.17.1-py3-none-any.whl.metadata (1.6 kB)
Using cached tensorflow_intel-2.17.0-cp312-cp312-win_amd64.whl (385.2 MB)
Using cached tensorboard-2.17.1-py3-none-any.whl (5.5 MB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow-intel
    Found existing installation: tensorflow_intel 2.18.0
    Uninstalling tensorflow_intel-2.18.0:
      Successfully uninstalled tensorflow_intel-2.18.0


  You can safely remove it manually.


In [4]:
tf.config.experimental_run_functions_eagerly(True)


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [5]:
!pip install tensorflow --upgrade

  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl (7.5 kB)
Using cached tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl (390.3 MB)
Using cached tensorboard-2.18.0-py3-none-any.whl (5.5 MB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow-intel
    Found existing installation: tensorflow-intel 2.17.0
    Uninstalling tensorflow-intel-2.17.0:
      Successfully uninstalled tensorflow-intel-2.17.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.0
    Uninstalling tensorflow-2.17.0:
      Successfully uninstalled tensorflow-2.17.0


In [ ]:
import os
import zipfile
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# Unzip the dataset
zip_file = "resized_data (1).zip"
extraction_path = "./resized_data"
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

# Define dataset directories and parameters
data_dir = extraction_path
batch_size = 32
img_height = 299
img_width = 299
num_classes = 4
epochs = 10

# Prepare K-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Prepare data augmentation
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    zoom_range=0.2,
    rotation_range=10,
    validation_split=0.2
)

# Load and preprocess images for k-fold
image_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

# Convert file paths to NumPy array for indexing
filepaths = np.array(image_generator.filepaths)

# K-fold cross-validation
fold_no = 1
accuracy_per_fold = []

for train_idx, val_idx in kf.split(filepaths):
    # Split file paths into training and validation sets
    train_filepaths = filepaths[train_idx]
    val_filepaths = filepaths[val_idx]

    # Extract class labels from file paths
    train_labels = [os.path.basename(os.path.dirname(path)) for path in train_filepaths]
    val_labels = [os.path.basename(os.path.dirname(path)) for path in val_filepaths]

    # Create DataFrames for train and validation data
    train_df = pd.DataFrame({"filename": train_filepaths, "class": train_labels})
    val_df = pd.DataFrame({"filename": val_filepaths, "class": val_labels})

    # Create train and validation generators
    train_gen = datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col="filename",
        y_col="class",
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )
    val_gen = datagen.flow_from_dataframe(
        dataframe=val_df,
        x_col="filename",
        y_col="class",
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    # Define the model
    base_model = InceptionV3(weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze base layers
    for layer in base_model.layers:
        layer.trainable = False

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train the model
    print(f"Training for fold {fold_no}...")
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        callbacks=[early_stopping],
        verbose=1
    )

    # Evaluate the model
    scores = model.evaluate(val_gen, verbose=0)
    print(f"Fold {fold_no} - Accuracy: {scores[1]*100:.2f}%")
    accuracy_per_fold.append(scores[1])

    fold_no += 1

# Calculate and display average accuracy across folds
print(f"Average Accuracy: {np.mean(accuracy_per_fold)*100:.2f}%")

# Plot training and validation accuracy for the last fold
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy (Last Fold)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


Found 35829 images belonging to 3 classes.
Found 28663 validated image filenames belonging to 4 classes.
Found 7166 validated image filenames belonging to 4 classes.
Training for fold 1...
Epoch 1/10


C:\Users\nikul\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


896/896 ━━━━━━━━━━━━━━━━━━━━ 1310s 1s/step - accuracy: 0.5870 - loss: 1.0269 - val_accuracy: 0.7230 - val_loss: 0.6265
Epoch 2/10
896/896 ━━━━━━━━━━━━━━━━━━━━ 1054s 1s/step - accuracy: 0.7075 - loss: 0.6558 - val_accuracy: 0.7402 - val_loss: 0.5859
Epoch 3/10
896/896 ━━━━━━━━━━━━━━━━━━━━ 1055s 1s/step - accuracy: 0.7410 - loss: 0.5890 - val_accuracy: 0.7423 - val_loss: 0.5915
Epoch 4/10
896/896 ━━━━━━━━━━━━━━━━━━━━ 1055s 1s/step - accuracy: 0.7524 - loss: 0.5655 - val_accuracy: 0.7523 - val_loss: 0.5511
Epoch 5/10
896/896 ━━━━━━━━━━━━━━━━━━━━ 1056s 1s/step - accuracy: 0.7633 - loss: 0.5331 - val_accuracy: 0.7548 - val_loss: 0.5533
Epoch 6/10
896/896 ━━━━━━━━━━━━━━━━━━━━ 1054s 1s/step - accuracy: 0.7725 - loss: 0.5139 - val_accuracy: 0.7632 - val_loss: 0.5227
Epoch 7/10
896/896 ━━━━━━━━━━━━━━━━━━━━ 1458s 2s/step - accuracy: 0.7899 - loss: 0.4829 - val_accuracy: 0.7589 - val_loss: 0.5738
Epoch 8/10
896/896 ━━━━━━━━━━━━━━━━━━━━ 1577s 2s/step - accuracy: 0.7938 - loss: 0.4730 - val_accurac